In [5]:
import cPickle as pickle
import numpy as np
from pprint import pprint

with open('thesis/results/features_2016-12-14_10-15-43.pkl') as f:
    data, configuration = pickle.load(f)

print "PROJECTS:"
for p_name in data.iterkeys():
    print p_name

print
print "CONFIG:"
pprint(configuration)

PROJECTS:
Sowbug3
Cam1
Camera3
Zebrafish

CONFIG:
{'rf_class_weight': 'balanced_subsample',
 'rf_criterion': 'entropy',
 'seed': 42,
 'test_split_method': 'random'}


In [6]:
p_names = filter(lambda x: x != 'layer', [p for p in data.iterkeys()])
print p_names
splits = filter(lambda x: x != 'layer', [s for s in data[p_names[0]].iterkeys()])
print splits
features = filter(lambda x: x!= 'layer', [f for f in data[p_names[0]][splits[0]].iterkeys()])
print features

# for p_name, data in data.iteritems():
#     print p_name

['Sowbug3', 'Cam1', 'Camera3', 'Zebrafish']
[0.95, 0.9, 0.8, 0.99]
['fm_lbp ', 'fm_colornames ', 'fm_idtracker_c_d50 ', 'fm_idtracker_i_d50 ', 'fm_basic ', 'fm_hog ']


In [11]:
from tabulate import tabulate
with open('thesis/out/tables/experiments_features_tables1.tex', 'w+') as out_f:
    splits = sorted(splits)
    for split in splits:
        # print "------- ", split, " -------"
        
        make_bold = []
        
        # +2 - caption + dataset size
        caption_offset = 2
        table = [[] for _ in range(len(features) + caption_offset)]
        i = 0
        table[i].append('')
        for p_name in p_names:
            table[i].append(p_name)
            make_bold.append(p_name)
        
        i += 1
        table[i].append('')
        for p_name in p_names:
            table[i].append(data[p_name][split][features[0]]['X_shape'][0])
            
        keys = []
        
        sample_sizes = []
        
        for feat in features:
            i += 1
            # remove fm_ p
            f = '_'.join(feat.split('_')[1:])
            
            sample_sizes.append(data[p_name][split][feat]['X_shape'][0])
            
            if f[:6] == 'idtrac':
                f = f[:-5]
                if f[-1] == 'c':
                    f = f[:-2]+' C'
                else:
                    f = f[:-2]+' I'
            
            table[i].append(f)
            make_bold.append(f)
    
            for p_name in p_names:
                it = data[p_name][split][feat]
                if isinstance(it, dict):
                    std = "{:.2f}".format(np.std(it['accuracy']) * 100)
                    s = "{:.1f}".format(np.mean(it['accuracy']) * 100) 
                    s += " ("+std+")"
                    table[i].append(s)
                    
        
        print
        from math import ceil
        num_runs = len(data[p_name][split][feat]['class_frequency'])
        num_regions = np.mean(sample_sizes)
        num_train = int((1-split)*num_regions)
        
        title = "Features performance, trained on {}\% (\#regions displayed under dataset names), randomly picked".format(int(100 - (100*split)))
        label = 'features_accuracy_'+str(int(split*100))
        
        
        
        ### best in col
        table = np.array(table)
        tab = table[2:, 1:].copy()
        for i in range(tab.shape[0]):
            for j in range(tab.shape[1]):
                tab[i, j] = float(tab[i, j].split(' ')[0]) 
        
        tab = np.asarray(tab, dtype=np.float)
        amax = np.argmax(tab, axis=0)
        
        best_ = [table[id_+caption_offset, i+1] for i, id_ in enumerate(amax)]
        make_bold.extend(best_)
        
        s = tabulate(table, tablefmt='latex')
        pre = '\\begin {table}[H] \\caption {'+title+'} \\label{tab:'+label+'} \\begin{center}'
        post = '\\end{center}\\end {table}'
        for t in make_bold:
            s = s.replace(t, '\\textbf{'+t+'}')
        s = pre + s + post
        
        out_f.write(s)
        out_f.write("\n\n")
            
        print s
        print


\begin {table}[H] \caption {Features performance, trained on 20\% (\#regions displayed under dataset names), randomly picked} \label{tab:features_accuracy_80} \begin{center}\begin{tabular}{lllll}
\hline
             & \textbf{Sowbug3}     & \textbf{Cam1}        & \textbf{Camera3}     & \textbf{Zebrafish}   \\
             & 8386        & 7639        & 11701       & 20925       \\
 \textbf{lbp }        & 69.8 (0.52) & 51.7 (0.66) & 39.9 (0.69) & 38.8 (0.33) \\
 \textbf{colornames } & 81.9 (0.67) & 78.8 (0.79) & 70.2 (0.42) & 59.3 (0.35) \\
 \textbf{idtracker C} & 79.2 (0.52) & 89.0 (0.35) & 82.5 (0.60) & 81.9 (0.42) \\
 \textbf{idtracker I} & 82.3 (0.57) & \textbf{91.9 (0.33)} & \textbf{84.8 (0.71)} & \textbf{87.9 (0.44)} \\
 \textbf{basic }      & \textbf{87.1 (0.45)} & 79.5 (0.43) & 64.2 (0.32) & 67.5 (0.36) \\
 \textbf{hog }        & 69.4 (0.65) & 51.2 (0.58) & 39.7 (0.40) & 39.1 (0.21) \\
\hline
\end{tabular}\end{center}\end {table}


\begin {table}[H] \caption {Features performanc